In [88]:
from __future__ import annotations

import numpy as np
import pandas as pd
import pandera.pandas as pa

from ban_carbon_research.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

def str_to_float(s: pd.Series[str]) -> pd.Series[np.float64]:
    return s.replace("No Data", np.nan).str.replace(",", "").astype(np.float64)

df = pd.read_csv(RAW_DATA_DIR / "rggi-emissions-annual-facility.csv")

numeric_columns = [
    "Op Time", "Op Hours", "CO2 Mass (Tons)", "Heat Input (mmBtu)",
    "Eligible Biomass (Tons) (State Value)",
    "Eligible CHP Thermal Output (Tons) (State Value)"
]

for c in numeric_columns:
    df[c] = str_to_float(df[c])

schema = pa.DataFrameSchema(
    {
        "Year": pa.Column(
            int,
            checks = pa.Check.between(2009, 2025),
            nullable = False
            ),
        "Source Name": pa.Column(
            str,
            checks = pa.Check.ne(""),
            nullable = False
            ),
        "ORIS Code": pa.Column(
            int,
            checks = pa.Check.gt(0),
            nullable = False
            ),
        "State": pa.Column(
            str,
            checks = pa.Check.isin(
                [
                    'CT', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI',
                    'VA', 'VT'
                    ]
                ),
            nullable = False
            ),
        **{
            c: pa.Column(np.float64, checks = pa.Check.ge(0), nullable = True)
            for c in numeric_columns
            },
        "Reporting Status": pa.Column(
            str,
            checks = pa.Check.isin(["Complete", "Incomplete"])
        )
    },
    checks = pa.Check(
        lambda df: df.duplicated(["Year", "ORIS Code"]).sum() == 0,
        name = "ORIS-year_is_unique_identifier"
    )
)
df = schema.validate(df)

df.to_csv(
    PROCESSED_DATA_DIR / "rggi-emissions-annual-facility.csv",
    index=False
    )

In [3]:
from __future__ import annotations

import numpy as np
import pandas as pd
import pandera.pandas as pa

import geopandas

from ban_carbon_research.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

fname = "Power_Plants"

gdf = geopandas.read_file(RAW_DATA_DIR / fname)

mw_columns = [
        "Install_MW", "Total_MW", "Bat_MW", "Bio_MW", "Coal_MW",
        "Geo_MW", "Hydro_MW", "HydroPS_MW", "NG_MW", "Nuclear_MW",
        "Crude_MW", "Solar_MW", "Wind_MW", "Other_MW"
        ]
# Check that there are no frequencies at suspicious/missing values like 999.
#[print(df[c].value_counts()) for c in mw_columns]

schema = pa.DataFrameSchema(
    {
        **{
            c: pa.Column(np.int32, unique = True, nullable = False)
            for c in ["OBJECTID", "Plant_Code"]
        },
        **{
            c: pa.Column(np.float64, checks = pa.Check.ge(0), nullable = False)
            for c in mw_columns
        },
        "geometry": pa.Column(nullable = False)
    },
)
gdf = schema.validate(gdf)

gdf.to_file(PROCESSED_DATA_DIR / fname)

IsADirectoryError: [Errno 21] Is a directory: '/home/fdvom/ban-carbon-research/data/processed/Power_Plants'

In [4]:
from __future__ import annotations

import numpy as np
import pandas as pd
import pandera.pandas as pa

import geopandas

from ban_carbon_research.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

fname = "geoBoundariesCGAZ_ADM0"

gdf = geopandas.read_file(RAW_DATA_DIR / fname)


#gdf.to_file(PROCESSED_DATA_DIR / fname)

gdf

,shapeGroup,shapeType,shapeName,geometry
0,AFG,ADM0,Afghanistan,"POLYGON ((74.88986 37.23409, 74.88962 37.23314..."
1,GBR,ADM0,United Kingdom,"MULTIPOLYGON (((33.01302 34.64377, 33.01275 34..."
2,ALB,ADM0,Albania,"POLYGON ((20.0789 42.5558, 20.07928 42.55546, ..."
3,DZA,ADM0,Algeria,"POLYGON ((8.64194 36.94096, 8.64195 36.9407, 8..."
4,USA,ADM0,United States,"MULTIPOLYGON (((-168.15791 -14.53181, -168.161..."
...,...,...,...,...
213,126,DISP,Sanafir & Tiran Is.,"MULTIPOLYGON (((34.69143 27.95899, 34.69176 27..."
214,127,DISP,Senkakus,"POLYGON ((123.47986 25.7507, 123.48736 25.7498..."
215,128,DISP,Spratly Is,"POLYGON ((114.33955 11.44331, 114.33958 11.44,..."
216,129,DISP,West Bank,"POLYGON ((35.555 32.38936, 35.55517 32.38919, ..."
